In [2]:
#20240326_딥러닝 프레임워크

In [3]:
'''
테스트의 중요성 : 버그를 예방할 수 있고 자동화해야 소프트웨어 품질을 유지할 수 있다.
단위테스트: 중간중간에 테스트를 집어넣음. 테스트 가능한 가장 작은 소프트웨어를 실행하여 예상대로 동작하는지 확인.
단위의 크기를 작게 설정해서 단위테스트를 최대한 간단하게 작성.
'''

'\n테스트의 중요성 : 버그를 예방할 수 있고 자동화해야 소프트웨어 품질을 유지할 수 있다.\n단위테스트: 중간중간에 테스트를 집어넣음. 테스트 가능한 가장 작은 소프트웨어를 실행하여 예상대로 동작하는지 확인.\n단위의 크기를 작게 설정해서 단위테스트를 최대한 간단하게 작성.\n'

In [4]:
import unittest

In [5]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

In [7]:
 def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

In [ ]:
'''
가변길이: 인수 또는 반환값을 수가 달라질 수 있다는 뜻. 여러개의 변수를 입력받는 함수와 출력이 여러개인 함수가있다.
가변길이의 입출력표현: 변수들을 리스트에 넣어서 처리, 하나의 인수만 받고 하나의 값만 반환.
인수와 반환값을 리스트로 변경하면 더 많이 넣을 수 있다.



In [47]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [48]:
#기존코드가 ㅎ 하나를 담을 수 있었다면 변수르 배열로두면 더 많이 넣을 수 있음.
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [49]:
#add 클래스 구현. 입력을 리스트로 받아서 튜플로 바꿈
'''
add클래스의 forward 메서드 구현. 인수화 반환값이 리스트 또는 튜플이며 인수는 변수가 두개 담긴 리스트이다. 결과를 반환할 때는 튜플을 반환함.
순전파에서 가변 길이 입출력처리: 입력을 리스트로 바꿔서 여러개의 변수를 다룸. 출력은 튜플로 바꿔서 여러 개의 변수에 대응할 수 있다.
개선점: 입력시에 변수를 리스트로 전달하도록 요전. 변환값도 튜플로 전달. 복잡하다.
'''
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  #인수와 반환값을 리스트로 변경. 리스트내포를 사용.
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [63]:
class Add(Function): #인수와 반환값이 리스트 또는 튜플이여야하는 것을 주의한다.
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )

In [64]:
xs = [Variable(np.array(2)), Variable(np.array(3))] #리스트로 준비
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

AttributeError: 'list' object has no attribute 'data'

In [54]:
import numpy as np

In [34]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x



In [55]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #함수를 호출할때 (*)표를 붙임. 언팩은 리스트의 원소를 낱개로 풀어서 전달하는 기법.
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [56]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [57]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y


    def backward(self, gy):
         return gy, gy


In [58]:
def add(x0, x1):
    return Add()(x0, x1) #add클래스 대신 add 함수를 사용함

In [59]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [ ]:
'''
가변길이 인수에 대응한 Add클래스의 역전파.
덧셈 계산그래프에서 순전파와 역전파: 순전파는 입력2 출력1, 역전파는 입력1출력2. 
덧셈의 역전파는 상류에서 흘러오는 미분값을 그대로 흘려보낸다.

다변수 함수: 입력 변수가 여러개인 함수. 
편미분: 다변수함수에서 하나의 입력 변수에만 주목하여 다른 변수는 상수로 취급하여 미분하는 것
'''

In [67]:
#Variable클래스를 여러개의 변수에 대응할 수 있도록 수정.
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] #출력변수인 outputs에 담겨있는 미분값들을 리스트에 포함
            gxs = f.backward(*gys) #함수 f의 역전파 호출
            if not isinstance(gxs, tuple): # backward 반환값이 튜플이 아니면 튜플로 변환.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx #역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장

            if x.creator is not None:
                funcs.append(x.creator)


In [68]:
#Square 클래스 가변 길이 입출력 지원하도록 개선
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data #square 클래스를 새로운 Variable과 Function클래스에 맞게 수정
        gx = 2 * x * gy
        return gx

In [69]:
def add(x0, x1):
    return Add()(x0, x1)

def square(x):
     return Square()(x)
    

In [70]:
#z = x^2 + y^2를 계산하는 코드
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

TypeError: Add.forward() takes 2 positional arguments but 3 were given

In [72]:
#문제점: 같은 변수 반복 사용시 발생되는 문제점 해결 방법
'''
미분값을 처음 설정하는 경우는 출력에서 전해지는 미분값을 그래도 대입.
처음 이후부터는 전달된 미분값을 더해주도록 수정
'''
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad = ', x.grad)


x.grad =  2.0
3.0


In [73]:
#역전파시 미분값을 더해주는 코드 문제점
'''
같은 변수를 사용하여 다른 계산을 할 경우 계산이 꼬이는 문제.
서로 다른 두가지 미분을 했으므로 두번째 계산값이 다르게 출력됨.
이는 서로 다른 두가지 미분 계산을 수행했지만 메모리를 절약하고자 Variable 인스턴스 x를 재사용하여 
첫번째 미분값과 두번째 미분값이 더해져서 생긴 문제이다.
Variable 클래스에 미분값을 초기화하는 cleargrad 메서드를 추가하면 해결할 수 있다.
'''
#문제점: 같은 변수 반복 사용시 발생되는 문제점 해결 방법
'''
미분값을 처음 설정하는 경우는 출력에서 전해지는 미분값을 그래도 대입.
처음 이후부터는 전달된 미분값을 더해주도록 수정
'''
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}{}은 지원하지 않습니다.'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('1. x.grad = ', x.grad)

x.cleargrad()
y = add(x, x)
y.backward()
print('2. x.grad = ', x.grad)

1. x.grad =  2.0
2. x.grad =  2.0


In [ ]:
#복잡한 계산 그래프(이론)
'''
복잡하게 연결된 역전파의 올바른순서
-> 변수를 반복해서 사용하면 역전파는 출력쪽에서 전파하는 미분값을 더해야함.
      / B-b \
x->A-a        D->y  일때 반드시 함수 B와 C의 역전파를 모두 끝내고나서 함수A를 역전파해야한다.
      \ C-c /
'''
#함수 우선순위
'''
함수 리스트에서 지금까지는 마지막 원소를 꺼냈지만 A와 B중에서 출력쪽에 더 가까운 B를 꺼낼 수 있어야한다->함수 우선순위 설정
순전파일때 함수가 변수를 만들어내며 이는 부모자식관계이다.
이 관계를 기준으로 함수와 변수의 세대를 기록하고 세대를 우선순위로 둔다.
(순전파에서 모든 변수와 함수의세대가 설정되고 역전파에서 세대순으로 꺼낸다)
역전파시 세대수가 큰 쪽부터 처리하면 부모보다 자식이 먼저 처리됨을 보장한다. 
'''


In [ ]:
#세대 추가
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 #세대 추가 후 0으로 초기화

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 #set_creator메서드 호출될 때 부모 함수의 세대보다 1만큼 큰 값을 설정

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [] #funcs = [self.creator] 부분  여기부터 ~add_func()로 변경.
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator) #func.append(x.creator)부분 변경


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) #입력변수가 둘 이상이라면 가장 큰 generation수 선택
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)